In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from itertools import combinations
import lxml
import traceback
import re
from bs4 import BeautifulSoup as bs
from thefuzz import fuzz

C:\Users\Joshualevy\Anaconda3\envs\ioCapture\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
authors_df = pd.read_csv('econlit_scopus_matching_out/JPE_author_abstract_funding.csv')

In [3]:
aer_df = pd.read_csv('econlit_scopus_matching_out/aer_econlit.csv')
eca_df = pd.read_csv('econlit_scopus_matching_out/eca_econlit.csv')
jpe_df = pd.read_csv('econlit_scopus_matching_out/jpe_econlit.csv')
qje_df = pd.read_csv('econlit_scopus_matching_out/qje_econlit.csv')
res_df = pd.read_csv('econlit_scopus_matching_out/res_econlit.csv')
rje_df = pd.read_csv('econlit_scopus_matching_out/rje_econlit.csv')

aer_df['publication'] = 'aer'
eca_df['publication'] = 'eca'
jpe_df['publication'] = 'jpe'
qje_df['publication'] = 'qje'
res_df['publication'] = 'res'
rje_df['publication'] = 'rje'


econlit_df = pd.concat([aer_df, eca_df, jpe_df, qje_df, res_df, rje_df], ignore_index=True)

In [8]:
# abstracts_df = authors_df[['doi', 'sc_title', 'sc_abstract_text']]
abstracts_df = econlit_df[['title', 'date', 'year', 'publication', 'abstract', 'jel_code', 'L_code', 'K_code', 'D4_code', 'O3_code', 'G34_code']]

abstracts_df['anti_trust_indicator'] = 0
abstracts_df['market_power_indicator'] = 0
abstracts_df['L4_code'] = 0
abstracts_df['K21_code'] = 0
abstracts_df['J3_code'] = 0



anti_trust_pattern = r'anti trust|anti-trust|antitrust'
market_power_pattern = r'market power|market-power'

C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_23112\1527440596.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abstracts_df['anti_trust_indicator'] = 0
C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_23112\1527440596.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abstracts_df['market_power_indicator'] = 0
C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_23112\1527440596.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [9]:
for row in range(0, len(abstracts_df)):
    # sc_abstract_text 

    abstract = abstracts_df.loc[row, 'abstract']
    publication = abstracts_df.loc[row, 'publication']
    if type(abstracts_df.loc[row, 'abstract']) !=str:
        continue
    if re.search(anti_trust_pattern, abstract, flags=re.I):
        # print("Anti-trust: {}".format(row))
        abstracts_df.loc[row, 'anti_trust_indicator']=1
        print('ANTI-TRUST--{} in abstract: {}'.format(publication, row))
    if re.search(market_power_pattern, abstract, flags=re.I):
        # print("Market power: {}".format(row))
        abstracts_df.loc[row, 'market_power_indicator']=1
        print('MARKET-POWER--{} in abstract: {}'.format(publication, row))
    # if re.search(r'merger', abstract, flags=re.I):
    #     print('MERGER--{} in abstract: {}'.format(publication, row))
    # if re.search(r'cartel', abstract, flags=re.I):
    #     print('CARTEL--{} in abstract: {}'.format(publication, row))
    # if re.search(r'monopol', abstract, flags=re.I):
    #     print('MONOPOL--{} in abstract: {}'.format(publication, row))
    # if re.search(r'deadweight loss|deadweight|dead-weight|dead weight', abstract, flags=re.I):
    #     print('DEADWEIGHT LOSS--{} in abstract: {}'.format(publication, row))
    # if re.search(r'anti compet|anti-compet|anticompet', abstract, flags=re.I):
    #     print('ANTI-COMPET--{} in abstract: {}'.format(publication, row))

    if type(abstracts_df.loc[row, 'jel_code']) != str:
        continue
    if re.search(r'L4', abstracts_df.loc[row, 'jel_code'], flags=re.I):
        abstracts_df.loc[row, 'L4_code'] = 1

    if re.search(r'K21', abstracts_df.loc[row, 'jel_code'], flags=re.I):
        abstracts_df.loc[row, 'K21_code'] = 1
    if re.search(r'J3', abstracts_df.loc[row, 'jel_code'], flags=re.I):
        abstracts_df.loc[row, 'J3_code'] = 1


    


        
# print('ANTI-TRUST COUNT: {}'.format(len(abstracts_df[abstracts_df.anti_trust_indicator==1])))
# print('MARKET POWER COUNT: {}'.format(len(abstracts_df[abstracts_df.market_power_indicator==1])))
# print('L4-AER CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='aer')])))
# print('K21-AER CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='aer')])))
# print('L4-ECA CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='eca')])))
# print('K21-ECA CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='eca')])))
# print('L4-jpe CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='jpe')])))
# print('K21-jpe CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='jpe')])))
# print('L4-qje CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='qje')])))
# print('K21-qje CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='qje')])))
# print('L4-res CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='res')])))
# print('K21-res CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='res')])))
# print('L4-rje CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='rje')])))
# print('K21-rje CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='rje')])))


MARKET-POWER--aer in abstract: 104
MARKET-POWER--aer in abstract: 223
MARKET-POWER--aer in abstract: 238
MARKET-POWER--aer in abstract: 320
MARKET-POWER--aer in abstract: 420
MARKET-POWER--aer in abstract: 431
MARKET-POWER--aer in abstract: 539
ANTI-TRUST--aer in abstract: 551
MARKET-POWER--aer in abstract: 806
MARKET-POWER--aer in abstract: 822
MARKET-POWER--aer in abstract: 831
ANTI-TRUST--aer in abstract: 1058
ANTI-TRUST--aer in abstract: 1101
MARKET-POWER--aer in abstract: 1343
MARKET-POWER--aer in abstract: 1539
ANTI-TRUST--aer in abstract: 1807
MARKET-POWER--aer in abstract: 1858
MARKET-POWER--aer in abstract: 2190
ANTI-TRUST--aer in abstract: 2230
MARKET-POWER--aer in abstract: 2432
MARKET-POWER--aer in abstract: 2441
MARKET-POWER--aer in abstract: 2848
ANTI-TRUST--aer in abstract: 3042
ANTI-TRUST--aer in abstract: 3054
MARKET-POWER--aer in abstract: 3276
ANTI-TRUST--aer in abstract: 3299
MARKET-POWER--aer in abstract: 3301
MARKET-POWER--aer in abstract: 3825
MARKET-POWER--aer i

In [10]:
abstracts_df

,title,date,year,publication,abstract,jel_code,L_code,K_code,D4_code,O3_code,G34_code,anti_trust_indicator,market_power_indicator,L4_code,K21_code,J3_code
0,Synthetic Difference-in-Differences,2021-12-01,2021,aer,We present a new estimator for causal effects ...,"['C23', 'H25', 'H71', 'I18', 'L66']",1,0,0,0,0,0,0,0,0,0
1,Delegation in Veto Bargaining,2021-12-01,2021,aer,A proposer requires a veto player's approval t...,"['D72', 'D82']",0,0,0,0,0,0,0,0,0,0
2,M Equilibrium: A Theory of Beliefs and Choices...,2021-12-01,2021,aer,"We introduce a set-valued solution concept, M ...","['C72', 'C90', 'D83']",0,0,0,0,0,0,0,0,0,0
3,Prep School for Poor Kids: The Long-Run Impact...,2021-12-01,2021,aer,This paper evaluates the long-run effects of H...,"['I21', 'I26', 'I28', 'I38', 'J24']",0,0,0,0,0,0,0,0,0,0
4,Team-Specific Human Capital and Team Performan...,2021-12-01,2021,aer,This paper studies whether team members' past ...,"['I10', 'J24', 'M12', 'M54']",0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,Welfare Effects of Changes in Environmental Qu...,1997-01-01,1997,rje,The authors adapt the theoretical state-prefer...,"['Q26', 'Q24']",0,0,0,0,0,0,0,0,0,0
15094,Market Segmentation and the Sources of Rents f...,1997-01-01,1997,rje,The authors evaluate the sources of transitory...,"['L63', 'L13', 'O31']",1,0,0,1,0,0,1,0,0,0
15095,Contract Form and Procurement Costs: The Impac...,1997-01-01,1997,rje,It is claimed that many regulatory rules enfor...,"['L74', 'K12', 'H57', 'L51']",1,1,0,0,0,0,0,0,0,0
15096,Articles in Honor of Richard E. Quandt: An Int...,1997-01-01,1997,rje,ECONLIT None Found,['B31'],0,0,0,0,0,0,0,0,0,0


In [ ]:
publications_year_df = abstracts_df.groupby(by=['publication', 'year']).agg(
    {
        'title' : lambda x: len(x)
    }
)
publications_year_df.reset_index(inplace=True)
publications_year_df.rename(columns={'title': 'count'}, inplace=True)
publications_year_df.to_csv('../adhoc_material/Reports/04-21/publications_year.csv', index=False, encoding='utf-8')


In [ ]:
theory_codes_pattern = r'C7|D11|D5|D21|D85|D86'

L_papers_df = abstracts_df[abstracts_df.L_code==1]
L_papers_df.reset_index(inplace=True)
L_papers_df['contains_theory'] = 0

for i in range(0, 10):
    generated_key = 'L' + str(i)+ '_code'
    L_papers_df[generated_key]=0

for j in [0,1,2,3,4,9]:
    generated_key = 'L4' + str(j) + '_code'
    L_papers_df[generated_key] = 0

for row in range(0,len(L_papers_df)):
    if re.search(theory_codes_pattern, L_papers_df.loc[row, 'jel_code'], flags=re.I):
        L_papers_df.loc[row, 'contains_theory'] =1

    for i in range(0, 10):
        generated_pattern = r'L'+ str(i)
        generated_column = 'L' + str(i) + '_code'
        if re.search(generated_pattern, L_papers_df.loc[row, 'jel_code'], flags=re.I):
            L_papers_df.loc[row, generated_column] = 1

    for j in [0,1,2,3,4,9]:
        generated_pattern = r'L4'+str(j)
        generated_column = 'L4' + str(j) + '_code'
        if re.search(generated_pattern, L_papers_df.loc[row, 'jel_code'], flags=re.I):
            L_papers_df.loc[row, generated_column] = 1
            print(generated_pattern)
            print(abstracts_df.loc[row, 'jel_code'])


print("TOTAL IO: {}".format(len(L_papers_df)))
print("TOTAL ANTITRUST: {}".format(len(L_papers_df[L_papers_df.L4_code ==1])))
print("IO and at least one theory : {}".format(len(L_papers_df[L_papers_df.contains_theory==1])))
print("ANTITRUST AND THEORY: {}".format(len(L_papers_df[(L_papers_df.contains_theory==1) & (L_papers_df.L4_code == 1)]))) 
## Plot these counts and shares by year

L_papers_df.to_csv('../adhoc_material/Reports/04-21/L_papers_indicators.csv', index=False, encoding='utf-8')
# L_papers_df

In [13]:
year_pub_count = pd.DataFrame()
year_pub_count['count'] = abstracts_df.groupby(by=['year', 'publication']).size()
year_pub_count.reset_index(inplace=True)
year_pub_count
indicator_count_df = abstracts_df.groupby(by=['year', 'publication']).agg({
    'L_code': lambda x: x.sum(),
    'K_code': lambda x: x.sum(),
    'L4_code': lambda x: x.sum(),
    'K21_code': lambda x: x.sum(),
    'D4_code': lambda x: x.sum(),
    'O3_code': lambda x: x.sum(),
    'G34_code': lambda x: x.sum(),
    'J3_code': lambda x: x.sum(),
    'anti_trust_indicator': lambda x: x.sum(),
    'market_power_indicator': lambda x: x.sum(),
})
indicator_count_df.reset_index(inplace=True)
indicator_count_df = pd.merge(indicator_count_df, year_pub_count, how='left', on=['year', 'publication'])
indicator_count_df.to_csv('indicator_count_df.csv', index=False, encoding='utf-8')
indicator_count_df

,year,publication,L_code,K_code,L4_code,K21_code,D4_code,O3_code,G34_code,J3_code,anti_trust_indicator,market_power_indicator,count
0,1990,aer,2,0,0,0,0,0,0,0,1,1,189
1,1990,eca,0,0,0,0,0,0,0,0,0,0,66
2,1990,jpe,1,0,0,0,0,0,0,0,0,1,73
3,1990,qje,1,0,0,0,0,0,0,0,0,0,54
4,1990,res,1,0,0,0,0,0,0,0,0,0,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2021,eca,12,7,0,4,15,4,1,5,0,2,109
188,2021,jpe,22,13,2,2,5,6,2,4,1,3,101
189,2021,qje,4,4,0,0,0,1,1,4,0,0,24
190,2021,res,3,0,0,0,1,2,0,1,0,0,28


In [ ]:
# aer_authors_df = pd.read_csv('econlit_scopus_matching_out/AER_author_abstract_funding.csv')
eca_authors_df = pd.read_csv('econlit_scopus_matching_out/ECA_author_abstract_funding.csv')
jpe_authors_df = pd.read_csv('econlit_scopus_matching_out/JPE_author_abstract_funding.csv')
qje_authors_df = pd.read_csv('econlit_scopus_matching_out/QJE_author_abstract_funding.csv')
res_authors_df = pd.read_csv('econlit_scopus_matching_out/RES_author_abstract_funding.csv')
rje_authors_df = pd.read_csv('econlit_scopus_matching_out/RJE_author_abstract_funding.csv')

authors_df = pd.concat([eca_authors_df, jpe_authors_df, qje_authors_df, res_authors_df, rje_authors_df], ignore_index=False)
authors_df = authors_df[['doi', 'sc_title', 'sc_pub_name', 'sc_author_given_name', 'sc_author_last_name', 'sc_author_id']]
authors_df.reset_index(inplace=True)

In [ ]:
for row in range(0, len(authors_df)):
    pub_name = authors_df.loc[row, 'sc_pub_name']
    if type(pub_name) != str:
        continue

    if pub_name == 'Econometrica : journal of the Econometric Society':
        authors_df.loc[row, 'sc_pub_name'] = 'Econometrica'
    elif pub_name == 'The Rand journal of economics':
        authors_df.loc[row, 'sc_pub_name'] = 'RAND Journal of Economics'

In [ ]:
authors_df.drop_duplicates(subset=['doi', 'sc_pub_name', 'sc_author_id'], inplace=True)

In [ ]:
# test_df = authors_df.head(100)
coauthors_df = authors_df.head(1000).groupby(by=['doi']).apply(lambda df: unique_author_pairs(df))
coauthors_df.reset_index(inplace=True)
coauthors_df[['author', 'coauthor']] = pd.DataFrame(coauthors_df['pairs'].tolist(), index=coauthors_df.index)
# coauthors_df

In [ ]:
def unique_author_pairs(sub_df):
    author_ids_list = sub_df['sc_author_id'].unique().tolist()
    doi = sub_df['doi'].unique()[0]
    
    if len(author_ids_list) ==1:
        unique_author_pairs = pd.Series(pair for pair in combinations([author_ids_list[0], np.nan], 2))
    else:
        unique_author_pairs = pd.Series(pair for pair in combinations(author_ids_list,2))
    # print(doi)
    # print(author_ids_list)
    # print(unique_author_pairs)
    
    # print('-----------------------------')
    return pd.DataFrame({
        # 'doi' : doi,
        'pairs' : unique_author_pairs
    })

In [ ]:
G = nx.from_pandas_edgelist(coauthors_df, 'author', 'coauthor')
# G = nx.from_pandas_adjacency(coauthors_df[['author', 'coauthor']])

G.remove_node(np.nan)
nan_edges = [edge for edge in G.edges if np.nan in edge]
print(nan_edges)
# G.remove_edges_from(nan_edges)
# for tup in G.edges:
#     if np.nan in tup:
#         source = tup[0]
#         target = tup[1]
#         print(tup)
#         G.remove_edge(source, target)
from matplotlib.pyplot import figure
figure(figsize=(20,15))
# nx.draw_shell(G, node_size=2)
# nx.draw_random(G, node_size=5)
# nx.draw_networkx_edges(G)
pos = nx.kamada_kawai_layout(G)
nx.draw(G, pos=pos)  # Draw the original graph